In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fb_master = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Data Merge (Regression, HR_Recovery, pyHRV)/Fitabase_combined_data_S01-S56.csv")
hr_aee = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Data Merge (Regression, HR_Recovery, pyHRV)/Fitabase_HR_aEE_MAX_S01-S56_08.17.2023.csv")
hr_recovery = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Data Merge (Regression, HR_Recovery, pyHRV)/HR_Recovery_S01-S56_08.21.2023.csv")
all_info = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Data Merge (Regression, HR_Recovery, pyHRV)/Wearables-REDCap-CRF_S01-S56.csv")

all_info.rename(columns = {'subject_id':'Subject_ID'}, inplace = True)


FileNotFoundError: ignored

#Adding New Columns Main dataset

In [ ]:
for i in list(hr_aee.columns):
  if i != "Condition" and i!="Patient_Id":
    fb_master[i] = ''



for i in list(hr_recovery.columns):
  if i != "Subject" and i!="Condition" and i!="Segment" and i!="Max Heart Rate":
    fb_master[i+"(mean)"] = ''
    fb_master[i+"(stdev)"] = ''



for i in list(all_info.columns):
  if i != "Subject_ID" and i!="Record ID:":
    fb_master[i] = ''


#Additional Functions

In [ ]:
import statistics as stat


def hr_aee_back(p_id, condition):
  li = []
  for i in list(hr_aee.columns):
    if i!="Patient_Id" and i!="Condition":
      li.append((i,list(hr_aee.query("Patient_Id == '"+p_id+"' and 	Condition == '"+condition+"'")[i])))
  return li


def hr_recovery_back(p_id,condition):
  li = []
  for i in list(hr_recovery.columns):
    if i != "Subject" and i!="Condition" and i!="Segment" and i!="Max Heart Rate":
      li.append((i+"(mean)",stat.mean(list(hr_recovery.query("Subject == '"+p_id+"' and 	Condition == '"+condition+"'")['1_min_HR_Recovery']))))
      li.append((i+"(stdev)",stat.pstdev(list(hr_recovery.query("Subject == '"+p_id+"' and 	Condition == '"+condition+"'")['1_min_HR_Recovery']))))
  return li



def all_info_back(p_id):
  li = []
  for i in list(all_info.columns):
    if i!="Subject_ID" and i!="Record ID:":
      li.append((i,list(all_info.query("Subject_ID == '"+p_id+"'")[i])))
  return li

#Engine

---



In [ ]:
hr_aee_done = []
all_info_done = []
hr_recov = []

for i in tqdm(range(fb_master.shape[0])):
  try:
  #if fb_master["Subject"][i] in hr_subject:
    if (fb_master["Subject"][i],fb_master["Condition"][i]) not in hr_aee_done:
      temp_hr_aee = hr_aee_back(fb_master["Subject"][i],fb_master["Condition"][i])
      hr_aee_done.append((fb_master["Subject"][i],fb_master["Condition"][i]))

    for j in temp_hr_aee:
      fb_master[j[0]][i] = j[1][0]


  #if fb_master["Subject"][i] in hr_recovery_subject:
    if (fb_master["Subject"][i],fb_master["Condition"][i]) not in hr_recov:
      temp_hr_recovery = hr_recovery_back(fb_master["Subject"][i],fb_master["Condition"][i])
      hr_recov.append((fb_master["Subject"][i],fb_master["Condition"][i]))

    for j in temp_hr_recovery:
      fb_master[j[0]][i] = j[1]


  #if fb_master["Subject"][i] in all_info_subject:
    if fb_master["Subject"][i] not in all_info_done:
      temp_all_info = all_info_back(fb_master["Subject"][i])
      all_info_done.append(fb_master["Subject"][i])

    for j in temp_all_info:
      fb_master[j[0]][i] = j[1][0]

  except:
    print("Something went worng!!!. But Do not worry, Engine is doing its work")
  finally:
    continue




In [ ]:
file_save_destination = '/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Data Merge (Regression, HR_Recovery, pyHRV)/'

fb_master.to_csv(file_save_destination+"Fitbit_feature_merged_version_S01-S56.csv",index=False)
